In [ ]:
%load_ext autoreload
%autoreload 2

##################################
##################################
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Imports
from pynattas.builder.netBuilder import GenericNetwork 
from pynattas.utils import layerCoder

import configparser
import pynattas as pnas
from datetime import datetime
import time

config = configparser.ConfigParser()
config.read('config.ini')


nas_check = config.getboolean(section='Mode', option='network_architecture_search')
ht_check = config.getboolean(section='Mode', option='hyperparameter_tuning')


task = config['Mode']['task']
if task not in ['C','D']:
    print(f"Error: Selected Task {task} is not available. Check config_optimizer.ini for verification.")
    exit()
else:
    print(f"Task: {task}")
# Define NAS parameters and run GA
max_layers = config.getint('GeneralNetwork', 'max_layers')
max_iterations = int(config['GA']['max_iterations'])
population_size = int(config['GA']['population_size'])
log_path = str(config['GA']['logs_dir_GA'])
mating_pool_cutoff = float(config['GA']['mating_pool_cutoff'])
mutation_probability = float(config['GA']['mutation_probability'])
############################################
print(f"Max Layers: {max_layers}")
print(f"Max Iterations: {max_iterations}")
print(f"Population Size: {population_size}")
print(f"Log Path: {log_path}")
print(f"Mating Pool Cutoff: {mating_pool_cutoff}")
print(f"Mutation Probability: {mutation_probability}")

In [ ]:
# Define population
from pynattas.builder.population import Population
import torch 

pop = Population(max_layers=max_layers, n_individuals=population_size)
print(f"Initial Population: {len(pop.population)}")
print(f"Current Generation: {pop.iteration}")

idx = 0
input_channels = 3
input_height = 224
input_width = 224
num_classes = 2

# def check_individual(individual):
#     try:
#         Net = GenericNetwork(
#                 individual.architecture, 
#                 input_channels,
#                 input_height,
#                 input_width,
#                 num_classes,)

#         Net.build()

#         x = torch.randn(1, 3, 224, 224)
#         Net(x)
#         model_size = Net.get_param_size()
#         valid = True
#     except Exception as e:
#         print(f"Error: {e}")
#         valid = False
#     finally:
#         # Reset memory
#         torch.cuda.empty_cache()
#         del Net
#         return valid


# for individual in pop.population:
#     print(f"Individual {idx}")
#     check_individual(individual)
#     idx += 1

In [ ]:
Net = GenericNetwork(
        individual.architecture, 
        input_channels,
        input_height,
        input_width,
        num_classes,)

Net.build()


x = torch.randn(1, 3, 224, 224)
Net(x)
print(Net.get_param_size())

In [ ]:
from pynattas.builder.individual import Individual

r = Individual(max_layers=5)

In [3]:
from pynattas.builder.population import Population
import torch 

max_layers = 5
population_size = 10

pop = Population(max_layers=max_layers, n_individuals=population_size)

In [ ]:
pop.population[0].architecture

In [6]:
# from pynattas.optimizers import ga

In [ ]:
# nas_result = pnas.optimizers.ga.ga_optimizer(
#     max_layers=max_layers,
#     max_iter=max_iterations,
#     n_individuals=population_size,
#     mating_pool_cutoff=mating_pool_cutoff,
#     mutation_probability=mutation_probability,
#     logs_directory=log_path,
#     task=task,
# )